In [1]:
from Bio import SeqIO
from esm.models.esm3 import ESM3 # ESM3 (seq-seq BERD-based LLM)
from esm.sdk.api import ESM3InferenceClient, ESMProtein, SamplingConfig
import numpy as np

PDB_file_path = '/p/vast1/bcwc/flavivirus/data/flavivirus_docking_results/dengue/denv2/4m9t/4m9t_moe_prep.pdb' 

seq = SeqIO.parse(PDB_file_path, 'pdb-seqres')

In [2]:
data = [(record.seq) for record in seq]

In [3]:
print(data)

[Seq('EADLELERAADVRWEEQAEISGSSPILSITISEDGSMSIKNEEEEQTLGELEDG...NTE')]


In [3]:
# Load ESM3 model
model: ESM3InferenceClient = ESM3.from_pretrained("esm3-open").to("cuda")

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

/usr/workspace/ranganath2/miniconda_p/envs/BioML/lib/python3.10/site-packages/esm/pretrained.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


In [4]:
# emb_list = []
sequence = data[0]
protein = ESMProtein(sequence=(str(sequence)))
protein_tensor = model.encode(protein)
protein_emb = model.forward_and_sample(protein_tensor, SamplingConfig(return_mean_embedding=True)).mean_embedding
protein_emb_np = protein_emb.detach().cpu().numpy()
print(sequence)
print(protein_emb_np.shape)
# emb_list.append(protein_emb_np/np.linalg.norm(protein_emb_np))

EADLELERAADVRWEEQAEISGSSPILSITISEDGSMSIKNEEEEQTLGELEDGAYRIKQKGILGYSQIGAGVYKEGTFHTMWHVTRGAVLMHKGKRIEPSWADVKKDLISYGGGWKLEGEWKEGEEVQVLALEPGKNPRAVQTKPGLFKTNTGTIGCVSLDFSPGTSGSPIVDKKGKVVGLYGNGVVTRSGAYVSAIANTE
(1536,)


In [5]:
norm_np = protein_emb_np/np.linalg.norm(protein_emb_np)

In [6]:
out_fn = "../denv2-4m9t.npy"
np.save(out_fn, norm_np)